In [1]:
basepath = 'C:/Users/shaur/Desktop/UTD/ANLP'

In [2]:
import sys
from pathlib import Path

In [3]:
base_folder = Path(basepath)
data_folder = base_folder/'datasets/emotion'
model_folder = base_folder/'models/nlp_spring_2024/emotion/nn'
custom_functions = base_folder/'custom_files'

In [4]:
sys.path.append(str(custom_functions))
model_folder.mkdir(exist_ok=True, parents = True)

In [5]:
import torch
import torch.nn as nn

# For working with textual data vocabularies and for displaying model summaries
from torchtext.vocab import vocab

# General-purpose Python libraries for random number generation and numerical operations
import random
import numpy as np

# Utilities for efficient serialization/deserialization of Python objects and for element tallying
import joblib
from collections import Counter
import re

# For creating lightweight attribute classes and for partial function application
from functools import partial

# For filesystem path handling, generating and displaying confusion matrices, and date-time manipulations
from sklearn.metrics import multilabel_confusion_matrix, hamming_loss, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from datetime import datetime
import pandas as pd
import emoji
import string

# For plotting and visualization
import matplotlib.pyplot as plt
import seaborn as sns

from kaggle import api

### NEW ##########################
# imports from Huggingface ecosystem
import evaluate
from datasets import load_dataset, DatasetDict, Dataset
from transformers import TrainingArguments, Trainer
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AutoConfig
from transformers import pipeline
import wandb

c:\Users\shaur\anaconda3\envs\anlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
train = pd.read_csv(data_folder/'train.csv')
test = pd.read_csv(data_folder/'test.csv')

In [7]:
train.drop(columns=['ID'], inplace=True)
train.head()

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0,0,0,1
1,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0
2,@Max_Kellerman it also helps that the majorit...,1,0,1,0,1,0,1,0,0,0,0
3,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0
4,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0,0,0,0


# Preprocessing

In [8]:
def get_emoji_regexp():
    emojis = sorted(emoji.EMOJI_DATA, key=len, reverse=True)
    pattern = u'(' + u'|'.join(re.escape(u) for u in emojis) + u')'
    return re.compile(pattern)

def strip_emoji(text):
    return re.sub(get_emoji_regexp(), r"", text)

def strip_all_entities(text): 
    text = text.replace('\r', '').replace('\n', ' ').replace('\n', ' ').lower()
    text = re.sub(r"(?:\@|https?\://)\S+", "", text) #remove links and mentions
    text = re.sub(r'[^\x00-\x7f]',r'', text) #remove non utf8/ascii characters such as '\x9a\x91\x97\x9a\x97'
    banned_list= string.punctuation + 'Ã'+'±'+'ã'+'¼'+'â'+'»'+'§'
    table = str.maketrans('', '', banned_list)
    text = text.translate(table)
    return text

def clean_hashtags(tweet):
    new_tweet = " ".join(word.strip() for word in re.split('#(?!(?:hashtag)\b)[\w-]+(?=(?:\s+#[\w-]+)*\s*$)', tweet)) #remove last hashtags
    new_tweet2 = " ".join(word.strip() for word in re.split('#|_', new_tweet)) #remove hashtags symbol from words in the middle of the sentence
    return new_tweet2

#Filter special characters such as & and $ present in some words
def filter_chars(a):
    sent = []
    for word in a.split(' '):
        if ('$' in word) | ('&' in word):
            sent.append('')
        else:
            sent.append(word)
    return ' '.join(sent)

def remove_mult_spaces(text): # remove multiple spaces
    return re.sub("\s\s+" , " ", text)

**Preprocess based on tweet length**

**Cross Validation**

In [9]:
train_clean = train['Tweet'].apply(strip_emoji).apply(strip_all_entities).apply(clean_hashtags).apply(filter_chars).apply(remove_mult_spaces)


In [10]:
train['Tweet'] = train_clean

In [11]:
X = train['Tweet'].values
y = train.loc[:, train.columns != 'Tweet'].values
y = np.array(y, dtype=float)

In [12]:
X_train, X_val, y_train, y_val  = train_test_split(X, y, random_state=42,test_size=0.1)

In [13]:
trainset = Dataset.from_dict({
    'texts': X_train,
    'labels': y_train
})
validset = Dataset.from_dict({
    'texts': X_val,
    'labels': y_val
})

In [14]:
trainset

Dataset({
    features: ['texts', 'labels'],
    num_rows: 6951
})

# Modelling

In [15]:
id2label = {0: "anger", 1: "anticipation", 2: "disgust", 3: "fear", 4: "joy", 5: "love", 6: "optimism", 7: "pessimism", 8: "sadness", 9: "surprise", 10: "trust"}
label2id = {id2label[k]: k for k in id2label}

In [16]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = nn.functional.sigmoid(torch.Tensor(logits))
    predictions = (predictions >= 0.5).int().numpy()

    overall_accuracy = accuracy_score(labels, predictions)
    overall_f1 = f1_score(labels, predictions, average='macro', zero_division=0)

    return {
        'eval_accuracy': overall_accuracy,
        'eval_f1': overall_f1
    }

## Distillbert

In [15]:
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [16]:
def tokenize_fn(batch):
    return tokenizer(text = batch["texts"], truncation=True)

In [17]:
trainset_tokenized = trainset.map(tokenize_fn, batched=True)
validset_tokenized = validset.map(tokenize_fn, batched=True)
trainset_tokenized = trainset_tokenized.remove_columns(['texts'])
validset_tokenized = validset_tokenized.remove_columns(['texts'])
trainset_tokenized.set_format(type='torch')
validset_tokenized.set_format(type='torch')

Map: 100%|██████████| 773/773 [00:00<00:00, 30689.25 examples/s]


In [20]:
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, 
    problem_type="multi_label_classification", 
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
training_args = TrainingArguments(
    num_train_epochs=3,

    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,  
    learning_rate=2e-5, 
    optim='adamw_torch',

    # Checkpoint saving and model evaluation settings
    output_dir=str(model_folder), 
    evaluation_strategy='steps', 
    eval_steps=100, 
    save_strategy="steps", 
    save_steps=100, 
    load_best_model_at_end=True, 
    save_total_limit=2, 
    metric_for_best_model="loss",
    greater_is_better=False, 
)

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset_tokenized,
    eval_dataset=validset_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

c:\Users\shaur\anaconda3\envs\anlp\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [24]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shauryat1298 (shauryat12). Use `wandb login --relogin` to force relogin


                                                  
  8%|▊         | 100/1305 [00:36<06:51,  2.93it/s]

{'eval_accuracy': 0.10996119016817593, 'eval_f1': 0.19404651334790357, 'eval_loss': 0.42624354362487793, 'eval_runtime': 2.3533, 'eval_samples_per_second': 328.479, 'eval_steps_per_second': 20.822, 'epoch': 0.23}


                                                  
 15%|█▌        | 200/1305 [01:13<06:14,  2.95it/s]

{'eval_accuracy': 0.17205692108667528, 'eval_f1': 0.27637320008856747, 'eval_loss': 0.38618406653404236, 'eval_runtime': 2.3257, 'eval_samples_per_second': 332.373, 'eval_steps_per_second': 21.069, 'epoch': 0.46}


...

                                                   
100%|█████████▉| 1300/1305 [08:00<00:01,  2.99it/s]

{'eval_accuracy': 0.24191461836998707, 'eval_f1': 0.4482068001912287, 'eval_loss': 0.32208094000816345, 'eval_runtime': 2.2987, 'eval_samples_per_second': 336.276, 'eval_steps_per_second': 21.316, 'epoch': 2.99}


100%|██████████| 1305/1305 [08:02<00:00,  2.70it/s]

{'train_runtime': 490.9241, 'train_samples_per_second': 42.477, 'train_steps_per_second': 2.658, 'train_loss': 0.32928414180360993, 'epoch': 3.0}


TrainOutput(global_step=1305, training_loss=0.32928414180360993, metrics={'train_runtime': 490.9241, 'train_samples_per_second': 42.477, 'train_steps_per_second': 2.658, 'train_loss': 0.32928414180360993, 'epoch': 3.0})

In [26]:
trainer.evaluate(trainset_tokenized)

100%|██████████| 435/435 [00:18<00:00, 23.44it/s]


{'eval_accuracy': 0.3195223708818875,
 'eval_f1': 0.5019881509278004,
 'eval_loss': 0.26109328866004944,
 'eval_runtime': 18.6019,
 'eval_samples_per_second': 373.671,
 'eval_steps_per_second': 23.385,
 'epoch': 3.0}

In [27]:
trainer.evaluate(validset_tokenized)

100%|██████████| 49/49 [00:02<00:00, 23.53it/s]


{'eval_accuracy': 0.24191461836998707,
 'eval_f1': 0.4482068001912287,
 'eval_loss': 0.32208094000816345,
 'eval_runtime': 2.1253,
 'eval_samples_per_second': 363.715,
 'eval_steps_per_second': 23.056,
 'epoch': 3.0}

## BERT Base Uncased

In [17]:
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [18]:
def tokenize_fn(batch):
    return tokenizer(text = batch["texts"], truncation=True)

In [19]:
trainset_tokenized = trainset.map(tokenize_fn, batched=True)
validset_tokenized = validset.map(tokenize_fn, batched=True)
trainset_tokenized = trainset_tokenized.remove_columns(['texts'])
validset_tokenized = validset_tokenized.remove_columns(['texts'])
trainset_tokenized.set_format(type='torch')
validset_tokenized.set_format(type='torch')

Map: 100%|██████████| 773/773 [00:00<00:00, 22309.82 examples/s]


In [20]:
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, 
    problem_type="multi_label_classification", 
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
training_args = TrainingArguments(
    num_train_epochs=3,

    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,  
    learning_rate=2e-5, 
    optim='adamw_torch',

    # Checkpoint saving and model evaluation settings
    output_dir=str(model_folder/'bert'), 
    evaluation_strategy='steps', 
    eval_steps=100, 
    save_strategy="steps", 
    save_steps=100, 
    load_best_model_at_end=True, 
    save_total_limit=2, 
    metric_for_best_model="loss",
    greater_is_better=False, 
)

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset_tokenized,
    eval_dataset=validset_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

c:\Users\shaur\anaconda3\envs\anlp\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [23]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shauryat1298 (shauryat12). Use `wandb login --relogin` to force relogin


                                                  
  8%|▊         | 100/1305 [01:03<11:50,  1.70it/s]

{'eval_accuracy': 0.11642949547218628, 'eval_f1': 0.2019508057022394, 'eval_loss': 0.41748642921447754, 'eval_runtime': 4.3605, 'eval_samples_per_second': 177.272, 'eval_steps_per_second': 11.237, 'epoch': 0.23}


                                                  
 15%|█▌        | 200/1305 [02:08<10:55,  1.69it/s]

{'eval_accuracy': 0.17723156532988357, 'eval_f1': 0.2537389419746899, 'eval_loss': 0.38499629497528076, 'eval_runtime': 4.3406, 'eval_samples_per_second': 178.087, 'eval_steps_per_second': 11.289, 'epoch': 0.46}


...

                                                   
100%|█████████▉| 1300/1305 [14:04<00:02,  1.71it/s]

{'eval_accuracy': 0.24708926261319533, 'eval_f1': 0.4587338686303221, 'eval_loss': 0.31893399357795715, 'eval_runtime': 4.3463, 'eval_samples_per_second': 177.851, 'eval_steps_per_second': 11.274, 'epoch': 2.99}


100%|██████████| 1305/1305 [14:10<00:00,  1.54it/s]

{'train_runtime': 859.935, 'train_samples_per_second': 24.25, 'train_steps_per_second': 1.518, 'train_loss': 0.3251488638106891, 'epoch': 3.0}


TrainOutput(global_step=1305, training_loss=0.3251488638106891, metrics={'train_runtime': 859.935, 'train_samples_per_second': 24.25, 'train_steps_per_second': 1.518, 'train_loss': 0.3251488638106891, 'epoch': 3.0})

In [24]:
trainer.evaluate(trainset_tokenized)

100%|██████████| 435/435 [00:35<00:00, 12.23it/s]


{'eval_accuracy': 0.3467127032081715,
 'eval_f1': 0.5303069348003384,
 'eval_loss': 0.24998746812343597,
 'eval_runtime': 35.6878,
 'eval_samples_per_second': 194.773,
 'eval_steps_per_second': 12.189,
 'epoch': 3.0}

In [25]:
trainer.evaluate(validset_tokenized)

100%|██████████| 49/49 [00:03<00:00, 12.44it/s]


{'eval_accuracy': 0.24708926261319533,
 'eval_f1': 0.4587338686303221,
 'eval_loss': 0.31893399357795715,
 'eval_runtime': 4.0667,
 'eval_samples_per_second': 190.079,
 'eval_steps_per_second': 12.049,
 'epoch': 3.0}

## Roberta Base

In [26]:
checkpoint = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

c:\Users\shaur\anaconda3\envs\anlp\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shaur\.cache\huggingface\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [27]:
def tokenize_fn(batch):
    return tokenizer(text = batch["texts"], truncation=True)

In [28]:
trainset_tokenized = trainset.map(tokenize_fn, batched=True)
validset_tokenized = validset.map(tokenize_fn, batched=True)
trainset_tokenized = trainset_tokenized.remove_columns(['texts'])
validset_tokenized = validset_tokenized.remove_columns(['texts'])
trainset_tokenized.set_format(type='torch')
validset_tokenized.set_format(type='torch')

Map: 100%|██████████| 773/773 [00:00<00:00, 48356.36 examples/s]


In [29]:
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, 
    problem_type="multi_label_classification", 
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [30]:
training_args = TrainingArguments(
    num_train_epochs=3,

    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,  
    learning_rate=2e-5, 
    optim='adamw_torch',

    # Checkpoint saving and model evaluation settings
    output_dir=str(model_folder/'bert'), 
    evaluation_strategy='steps', 
    eval_steps=100, 
    save_strategy="steps", 
    save_steps=100, 
    load_best_model_at_end=True, 
    save_total_limit=2, 
    metric_for_best_model="loss",
    greater_is_better=False, 
)

In [31]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset_tokenized,
    eval_dataset=validset_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

c:\Users\shaur\anaconda3\envs\anlp\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [32]:
trainer.train()

                                                  
  8%|▊         | 100/1305 [01:08<12:59,  1.55it/s]

{'eval_accuracy': 0.12807244501940493, 'eval_f1': 0.2257423799023587, 'eval_loss': 0.4027841091156006, 'eval_runtime': 4.4292, 'eval_samples_per_second': 174.524, 'eval_steps_per_second': 11.063, 'epoch': 0.23}


                                                  
 15%|█▌        | 200/1305 [02:19<11:47,  1.56it/s]

{'eval_accuracy': 0.2056921086675291, 'eval_f1': 0.35617407297684045, 'eval_loss': 0.3598071038722992, 'eval_runtime': 4.4221, 'eval_samples_per_second': 174.802, 'eval_steps_per_second': 11.081, 'epoch': 0.46}


                                                  
 23%|██▎       | 300/1305 [03:29<10:36,  1.58it/s]

...

                                                   
100%|█████████▉| 1300/1305 [15:10<00:03,  1.57it/s]

{'eval_accuracy': 0.2613195342820181, 'eval_f1': 0.49549515494873986, 'eval_loss': 0.30794021487236023, 'eval_runtime': 4.3624, 'eval_samples_per_second': 177.195, 'eval_steps_per_second': 11.232, 'epoch': 2.99}


100%|██████████| 1305/1305 [15:15<00:00,  1.43it/s]

{'train_runtime': 915.7151, 'train_samples_per_second': 22.772, 'train_steps_per_second': 1.425, 'train_loss': 0.31927538758493473, 'epoch': 3.0}


TrainOutput(global_step=1305, training_loss=0.31927538758493473, metrics={'train_runtime': 915.7151, 'train_samples_per_second': 22.772, 'train_steps_per_second': 1.425, 'train_loss': 0.31927538758493473, 'epoch': 3.0})

In [33]:
trainer.evaluate(trainset_tokenized)

100%|██████████| 435/435 [00:34<00:00, 12.46it/s]


{'eval_accuracy': 0.33505970363976406,
 'eval_f1': 0.5472622066111421,
 'eval_loss': 0.2523092031478882,
 'eval_runtime': 34.996,
 'eval_samples_per_second': 198.623,
 'eval_steps_per_second': 12.43,
 'epoch': 3.0}

In [34]:
trainer.evaluate(validset_tokenized)

100%|██████████| 49/49 [00:03<00:00, 12.69it/s]


{'eval_accuracy': 0.2613195342820181,
 'eval_f1': 0.49549515494873986,
 'eval_loss': 0.30794021487236023,
 'eval_runtime': 3.9362,
 'eval_samples_per_second': 196.382,
 'eval_steps_per_second': 12.449,
 'epoch': 3.0}

## Albert Base V2

In [18]:
checkpoint = "albert-base-v2"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

c:\Users\shaur\anaconda3\envs\anlp\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shaur\.cache\huggingface\hub\models--albert-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [19]:
def tokenize_fn(batch):
    return tokenizer(text = batch["texts"], truncation=True)

In [20]:
trainset_tokenized = trainset.map(tokenize_fn, batched=True)
validset_tokenized = validset.map(tokenize_fn, batched=True)
trainset_tokenized = trainset_tokenized.remove_columns(['texts'])
validset_tokenized = validset_tokenized.remove_columns(['texts'])
trainset_tokenized.set_format(type='torch')
validset_tokenized.set_format(type='torch')

Map: 100%|██████████| 773/773 [00:00<00:00, 21448.21 examples/s]


In [21]:
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, 
    problem_type="multi_label_classification", 
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
training_args = TrainingArguments(
    num_train_epochs=3,

    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,  
    learning_rate=2e-5, 
    optim='adamw_torch',

    # Checkpoint saving and model evaluation settings
    output_dir=str(model_folder/'bert'), 
    evaluation_strategy='steps', 
    eval_steps=100, 
    save_strategy="steps", 
    save_steps=100, 
    load_best_model_at_end=True, 
    save_total_limit=2, 
    metric_for_best_model="loss",
    greater_is_better=False, 
)

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset_tokenized,
    eval_dataset=validset_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

c:\Users\shaur\anaconda3\envs\anlp\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [24]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shauryat1298 (shauryat12). Use `wandb login --relogin` to force relogin


                                                  
  8%|▊         | 100/1305 [00:47<08:13,  2.44it/s]

{'eval_accuracy': 0.12807244501940493, 'eval_f1': 0.22535316264318997, 'eval_loss': 0.4130074381828308, 'eval_runtime': 6.0863, 'eval_samples_per_second': 127.007, 'eval_steps_per_second': 8.051, 'epoch': 0.23}


                                                  
 15%|█▌        | 200/1305 [01:35<07:27,  2.47it/s]

{'eval_accuracy': 0.1759379042690815, 'eval_f1': 0.29899779084443245, 'eval_loss': 0.3931446671485901, 'eval_runtime': 6.0939, 'eval_samples_per_second': 126.848, 'eval_steps_per_second': 8.041, 'epoch': 0.46}


...

                                                   
100%|█████████▉| 1300/1305 [10:12<00:02,  2.45it/s]

{'eval_accuracy': 0.26390685640362227, 'eval_f1': 0.4629353959323966, 'eval_loss': 0.31694474816322327, 'eval_runtime': 6.0877, 'eval_samples_per_second': 126.976, 'eval_steps_per_second': 8.049, 'epoch': 2.99}


100%|██████████| 1305/1305 [10:14<00:00,  2.12it/s]

{'train_runtime': 622.6521, 'train_samples_per_second': 33.491, 'train_steps_per_second': 2.096, 'train_loss': 0.3300281802356472, 'epoch': 3.0}


TrainOutput(global_step=1305, training_loss=0.3300281802356472, metrics={'train_runtime': 622.6521, 'train_samples_per_second': 33.491, 'train_steps_per_second': 2.096, 'train_loss': 0.3300281802356472, 'epoch': 3.0})

In [25]:
trainer.evaluate(trainset_tokenized)

100%|██████████| 435/435 [00:54<00:00,  7.99it/s]


{'eval_accuracy': 0.32297511149474895,
 'eval_f1': 0.511961651962669,
 'eval_loss': 0.2644408345222473,
 'eval_runtime': 54.5441,
 'eval_samples_per_second': 127.438,
 'eval_steps_per_second': 7.975,
 'epoch': 3.0}

In [26]:
trainer.evaluate(validset_tokenized)

100%|██████████| 49/49 [00:05<00:00,  8.37it/s]


{'eval_accuracy': 0.26390685640362227,
 'eval_f1': 0.4629353959323966,
 'eval_loss': 0.31694474816322327,
 'eval_runtime': 5.9872,
 'eval_samples_per_second': 129.109,
 'eval_steps_per_second': 8.184,
 'epoch': 3.0}

## Deberta V3 Large

In [18]:
checkpoint = "microsoft/deberta-v3-base"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

c:\Users\shaur\anaconda3\envs\anlp\lib\site-packages\transformers\convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [19]:
def tokenize_fn(batch):
    return tokenizer(text = batch["texts"], truncation=True, max_length=512)

In [20]:
trainset_tokenized = trainset.map(tokenize_fn, batched=True)
validset_tokenized = validset.map(tokenize_fn, batched=True)
trainset_tokenized = trainset_tokenized.remove_columns(['texts'])
validset_tokenized = validset_tokenized.remove_columns(['texts'])
trainset_tokenized.set_format(type='torch')
validset_tokenized.set_format(type='torch')

Map: 100%|██████████| 773/773 [00:00<00:00, 41403.14 examples/s]


In [21]:
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, 
    problem_type="multi_label_classification", 
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

c:\Users\shaur\anaconda3\envs\anlp\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shaur\.cache\huggingface\hub\models--microsoft--deberta-v3-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at mic

In [22]:
training_args = TrainingArguments(
    num_train_epochs=3,

    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,  
    learning_rate=2e-5, 
    optim='adamw_torch',

    # Checkpoint saving and model evaluation settings
    output_dir=str(model_folder/'bert'), 
    evaluation_strategy='steps', 
    eval_steps=100, 
    save_strategy="steps", 
    save_steps=100, 
    load_best_model_at_end=True, 
    save_total_limit=2, 
    metric_for_best_model="loss",
    greater_is_better=False, 
)

In [23]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset_tokenized,
    eval_dataset=validset_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

c:\Users\shaur\anaconda3\envs\anlp\lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [24]:
trainer.train()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shauryat1298 (shauryat12). Use `wandb login --relogin` to force relogin


                                                  
  8%|▊         | 100/1305 [01:56<21:41,  1.08s/it]

{'eval_accuracy': 0.062095730918499355, 'eval_f1': 0.17225961192325134, 'eval_loss': 0.42655500769615173, 'eval_runtime': 8.7874, 'eval_samples_per_second': 87.966, 'eval_steps_per_second': 5.576, 'epoch': 0.23}


                                                    
 15%|█▌        | 200/1305 [03:54<19:50,  1.08s/it]

{'eval_accuracy': 0.17852522639068563, 'eval_f1': 0.2522879002203507, 'eval_loss': 0.3916710913181305, 'eval_runtime': 8.7447, 'eval_samples_per_second': 88.396, 'eval_steps_per_second': 5.603, 'epoch': 0.46}


...

                                                   
100%|█████████▉| 1300/1305 [25:40<00:05,  1.07s/it]

{'eval_accuracy': 0.24191461836998707, 'eval_f1': 0.464928618741709, 'eval_loss': 0.31953272223472595, 'eval_runtime': 8.8618, 'eval_samples_per_second': 87.228, 'eval_steps_per_second': 5.529, 'epoch': 2.99}


100%|██████████| 1305/1305 [25:49<00:00,  1.19s/it]

{'train_runtime': 1556.8584, 'train_samples_per_second': 13.394, 'train_steps_per_second': 0.838, 'train_loss': 0.33648407450124224, 'epoch': 3.0}


TrainOutput(global_step=1305, training_loss=0.33648407450124224, metrics={'train_runtime': 1556.8584, 'train_samples_per_second': 13.394, 'train_steps_per_second': 0.838, 'train_loss': 0.33648407450124224, 'epoch': 3.0})

In [25]:
trainer.evaluate(trainset_tokenized)

100%|██████████| 435/435 [01:12<00:00,  5.96it/s]


{'eval_accuracy': 0.3100273341965185,
 'eval_f1': 0.5101161867221161,
 'eval_loss': 0.26065361499786377,
 'eval_runtime': 73.1274,
 'eval_samples_per_second': 95.053,
 'eval_steps_per_second': 5.949,
 'epoch': 3.0}

In [26]:
trainer.evaluate(validset_tokenized)

100%|██████████| 49/49 [00:07<00:00,  6.15it/s]


{'eval_accuracy': 0.24191461836998707,
 'eval_f1': 0.464928618741709,
 'eval_loss': 0.31953272223472595,
 'eval_runtime': 8.1392,
 'eval_samples_per_second': 94.973,
 'eval_steps_per_second': 6.02,
 'epoch': 3.0}

# Evaluation

In [75]:
checkpoint = "cardiffnlp/twitter-roberta-base-emotion-multilabel-latest"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

c:\Users\shaur\anaconda3\envs\anlp\lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shaur\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-emotion-multilabel-latest. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [76]:
def tokenize_fn(batch):
    return tokenizer(text = batch["texts"], truncation=True, max_length=512)

In [77]:
trainset_tokenized = trainset.map(tokenize_fn, batched=True)
validset_tokenized = validset.map(tokenize_fn, batched=True)
trainset_tokenized = trainset_tokenized.remove_columns(['texts'])
validset_tokenized = validset_tokenized.remove_columns(['texts'])
trainset_tokenized.set_format(type='torch')
validset_tokenized.set_format(type='torch')

Map: 100%|██████████| 773/773 [00:00<00:00, 20781.32 examples/s]


In [78]:
model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint, 
    problem_type="multi_label_classification", 
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id
)

In [79]:
training_args = TrainingArguments(
    num_train_epochs=3,

    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    optim='adamw_torch',

    # Checkpoint saving and model evaluation settings
    output_dir=str(model_folder/'roberta_base_emotion'), 
    evaluation_strategy='steps', 
    eval_steps=100, 
    save_strategy="steps", 
    save_steps=100, 
    load_best_model_at_end=True, 
    save_total_limit=2, 
    metric_for_best_model="loss",
    greater_is_better=False, 
)

In [81]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=trainset_tokenized,
    eval_dataset=validset_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [82]:
trainer.train()

                                                  
  8%|▊         | 100/1305 [01:05<12:11,  1.65it/s]

{'eval_accuracy': 0.2949547218628719, 'eval_f1': 0.5780410973159245, 'eval_loss': 0.268928587436676, 'eval_runtime': 4.3623, 'eval_samples_per_second': 177.202, 'eval_steps_per_second': 11.233, 'epoch': 0.23}


                                                  
 15%|█▌        | 200/1305 [02:11<11:09,  1.65it/s]

{'eval_accuracy': 0.31565329883570503, 'eval_f1': 0.5934336355545805, 'eval_loss': 0.26409852504730225, 'eval_runtime': 4.3488, 'eval_samples_per_second': 177.751, 'eval_steps_per_second': 11.268, 'epoch': 0.46}


                                                  
 23%|██▎       | 300/1305 [03:18<10:02,  1.67it/s]

...

                                                   
100%|█████████▉| 1300/1305 [14:24<00:02,  1.67it/s]

{'eval_accuracy': 0.35834411384217335, 'eval_f1': 0.6631309024646961, 'eval_loss': 0.25456538796424866, 'eval_runtime': 4.3392, 'eval_samples_per_second': 178.142, 'eval_steps_per_second': 11.292, 'epoch': 2.99}


100%|██████████| 1305/1305 [14:28<00:00,  1.50it/s]

{'train_runtime': 868.9618, 'train_samples_per_second': 23.998, 'train_steps_per_second': 1.502, 'train_loss': 0.22507325972633802, 'epoch': 3.0}


TrainOutput(global_step=1305, training_loss=0.22507325972633802, metrics={'train_runtime': 868.9618, 'train_samples_per_second': 23.998, 'train_steps_per_second': 1.502, 'train_loss': 0.22507325972633802, 'epoch': 3.0})